In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

l_filenames = []

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        l_filenames.append(os.path.join(dirname, filename))
        

        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Facebook V: Predicting Check Ins

Facebook created an artificial world consisting of more than 100,000 places located in a 10 km by 10 km square. Data was fabricated to resemble location signals coming from mobile devices, giving you a flavor of what it takes to work with real data complicated by inaccurate and noisy values. Inconsistent and erroneous location data can disrupt experience for services like Facebook Check In.

## The task

Build a model that shall predict which business a user is checking into based on their location, accuracy, and timestamp. 
For a given set of coordinates, the task is to return a ranked list of the most likely places the customers checked in. 

## The data sets

train.csv, test.csv 
- row_id: id of the check-in event
- x y: coordinates
- accuracy: location accuracy 
- time: timestamp
- place_id: id of the business, this is the target you are predicting

sample_submission.csv - a sample submission file in the correct format with random predictions

## Further notes

In the competition the participants were given around 30 million (simulated) check-ins on Facebook in a 10km by 10km grid. The goal is to build a model that predicts what business a user checks into based on spatial and temporal information. The difficult part is that there are around 100.000 different classes (place_id’s) so most supervised learning techniques won’t work on the entire dataset. However most classes are clustered in only certain parts of the grid so the idea is to select a sub-square within the grid and try to classify within small square. First I’ll do some exploratory data analysis in the smaller square then I’ll use a random forest algorithm for prediction and finally, I’ll analyze the results.

## Approach:

1. Exploratory data analysis in the sub-square
2. Use of different classification algorithms for prediction 
3. Analysis of the results




# 1. Exploratory data anylsis in sub-square

In [ ]:
# load datasets
df_train = pd.read_csv(str(l_filenames[0]))
df_test = pd.read_csv(str(l_filenames[2]))
df_sample=pd.read_csv(str(l_filenames[1]))


In [ ]:
# import all modules needed in the following
import tensorflow
import keras
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
# inspect and clean datasets

df_train = df_train[((df_train["x"] != np.NaN) | (df_train["x"] != 0)) & ((df_train["y"] != np.NaN) | (df_train["y"] != 0))]
print(df_train)


# print max and min of each values
print("Range of x: ", df_train.x.max(), df_train.x.min())
print("Range of y: ",df_train.y.max(), df_train.y.min())
print("Range of time: ",df_train.time.max(), df_train.time.min())
print("Range of place_id: ",df_train.place_id.max(), df_train.place_id.min())


## Data cleaning and selection

The data set is very large (29.1 Million check ins). For testing methods I randomly choose a subset of the data based on the location (x, and y coordinates) of the samples. I will randomly choose 

In addition, the time column (which is currently numeric) shall be transformed to hour, day, week, month, and year.


In [ ]:
# selection of sub-square of the datat grid. Maximum in x and y coordinates is 10, minimum is 0
df_sub = df_train[((df_train.loc[:,"x"] > 2.5) & (df_train.loc[:,"x"] < 2.75)) &
                  ((df_train.loc[:,"y"] > 2.5) & (df_train.loc[:,"y"] < 2.75))]

# transformation to extract temporal features
df_sub.loc[:,"hour"] =    (df_sub.loc[:,"time"]/60).round().mod(24).astype(int)
df_sub.loc[:,"weekday"] = (df_sub.loc[:,"time"]/(60*24)).round().mod(7).astype(int)
df_sub.loc[:,"month"] =   (df_sub.loc[:,"time"]/(60*24*30)).round().mod(12).astype(int) #month-ish
df_sub.loc[:,"year"] =    (df_sub.loc[:,"time"]/(60*24*365)).astype(int)
df_sub.loc[:,"day"] =     (df_sub.loc[:,"time"]/(60*24)).round().mod(365).astype(int)

# number of different businesses
print("Distinct labels: ", df_sub.place_id.nunique())

## Preliminary plot of the data

In [ ]:
# scatterplot of classes  (businesses) within the subgrid
sns.set(rc={'figure.figsize':(15,15)})
sns.scatterplot(data=df_sub, x="x", y="y", hue="place_id", palette = "Accent", legend = False)

It is visible that in the classes in the data subset are not well separated.

Let us now introduce the time feature to the scatterplot and transform it to a 3d plot.

I will try to use the *time* column, followed by the distinct time features that have been created above: **hour**, **minute**, and **day**

In [ ]:
# scatterplot of classes  (businesses) within the subgrid
sns.set(rc={'figure.figsize':(18,20)})
#sns.set(style = "darkgrid")

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

x = df_sub['x']
y = df_sub['y']
z = df_sub['time']
c = df_sub["place_id"]

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("time")

ax.scatter(x, y, z, c=c, cmap = "hsv")

plt.show()

The 3d plot shows a more distinct distribution of classes. However, it is still visible that the classes are not at all separated.

Lets try it with the **hour** as well as **weekday** feature, to see if the classes seem better separated when the examples are arranged based on the hour or the wekday of visit.



In [ ]:
# scatterplot of classes  (businesses) within the subgrid
sns.set(rc={'figure.figsize':(18,20)})
#sns.set(style = "darkgrid")

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

x = df_sub['x']
y = df_sub['y']
z = df_sub['hour']
c = df_sub["place_id"]

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("time")

ax.scatter(x, y, z, c=c, cmap = "hsv")

plt.show()

In [ ]:
# scatterplot of classes  (businesses) within the subgrid
sns.set(rc={'figure.figsize':(18,20)})
#sns.set(style = "darkgrid")

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

x = df_sub['x']
y = df_sub['y']
z = df_sub['time']
c = df_sub["weekday"]

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("time")

ax.scatter(x, y, z, c=c, cmap = "hsv")

plt.show()

It is visible that the weekday feature leads to much less visible groups of data points from either classes, compared to the hour feature.

Let us have a quick look at only classes that have a minimum of **n assignments**. This is important because some classes only have very few examples assigned to them, which might distort the classification.

For this example, I will only print the class assignments for classes with **minimum 200 examples being assigned to them.**

In [ ]:
# scatterplot of classes  (businesses) within the subgrid
sns.set(rc={'figure.figsize':(18,20)})
#sns.set(style = "darkgrid")
place_counts = df_sub.place_id.value_counts()
mask = (place_counts[df_sub.place_id.values] >= 200).values
df_only = df_sub.loc[mask]

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

x = df_only['x']
y = df_only['y']
z = df_only['hour']
c = df_only["place_id"]

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("time")

ax.scatter(x, y, z, c=c, cmap = "hsv")

plt.show()

### Result from preliminary plots

The preliminary scatterplots show no clear decision boundary. The classes are not well separated on a 2-dimensional level.

However, adding a temporal feature, it is visible that especially on the hourly resolution, there are some classes which are more present in certain hours.

## Accuracy distribution

The accuracy of the class assignment of the check ins given in the dataset isn’t exactly defined. From first principles, we could think of it a few ways -

- Error on some arbitary scale. This seems unlikely, since the max is > 1,000.
- Error on the same scale as x and y. Now, this could be an estimated radius (with the x and y values as the center); either normal or squared.

Plot the distribution of accuracy aswell as a default kernel density estimation plot:


In [ ]:
# histogramm & KDE of the accuracy feature
sns.set(rc={'figure.figsize':(15,15)})
sns.displot(data = df_sub, x="accuracy", bins=50)
sns.displot(data=df_sub, x="accuracy", kind="kde")

It looks like we have three peaks (two peaks if we exclude the examples with an accuracy value of 0. Are there underlying parameters in the simulation at these peaks?

We might also think that there are different measures of accuracy at different locations.

Let’s see how even the accuracy is distributed over x and y.


In [ ]:
# scatterplot of the accuracy in 3d scale
sns.set(rc={'figure.figsize':(18,20)})
#sns.set(style = "darkgrid")

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

x = df_sub['x']
y = df_sub['y']
z = df_sub['accuracy']

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("time")

ax.scatter(x, y, z,color = "blue")

plt.show()

The accuracy distribution seems visually random with a few accuracy outliers and overall low values.

Let us look at the outliers and if there are centers of these outliers. Outliers are hereby defined as examples with accuracy > 250.



In [ ]:
# scatterplot of classes  (businesses) within the subgrid
sns.set(rc={'figure.figsize':(15,15)})
sns.scatterplot(data=df_sub[df_sub.accuracy >250], x="x", y="y", hue="accuracy", palette = "Accent", legend = False)

### Results of the accuracy analysis:

There are no centers of particularly high / low accuracy of the class assignments in the data on geographic scale.

Given the interpretation of the accuracy, it could be that the examples with high accuracy rating (= higher probability of the label assignment being wrong) distort the quality of the classification training and can lead to a model that systematically misclassifies examples. 

This will later be tested via a grid search using a threshold to **clude training examples** with an accuracy **above**his certain threshold.  


## Time distribution

Let us now investigate the distribution of the time features **hour**, **weekday**, and **day** of the year, to investigate if there are significant peaks in the data

In [ ]:
# histogramm & KDE of the time features:
sns.set(rc={'figure.figsize':(15,15)})

# hour
sns.displot(data = df_sub, x="hour", bins=24)
sns.displot(data=df_sub, x="hour", kind="kde")

# weekday
sns.displot(data = df_sub, x="weekday", bins=7)
sns.displot(data=df_sub, x="weekday", kind="kde")

# day
sns.displot(data = df_sub, x="day", bins=24)
sns.displot(data=df_sub, x="day", kind="kde")

While there is no specific distribution recognizable in the weekday feature (similar number of sign ins per weekday) there is a peak in the afternoon hours recognizable as well as a higher number of sign ins in the first half of the year.


## Place ID Distribution

Now that we have some background on time, accuracy and (sideways) on x and y let’s look at place_id.

First, a quick check. Sometimes ID values aren’t really uniformly distributed.

In [ ]:
# histogramm & KDE of the time features:
sns.set(rc={'figure.figsize':(15,15)})

# hour
sns.displot(data = df_sub, x="place_id", bins=50)
sns.displot(data=df_sub, x="place_id", kind="kde", bw_adjust=1.5)

The distribution clearly has two peaks, looking like a bimodal distribution. There seem to be certain places that have particularly high amounts of sign ins.

Let us group the data by the place ID and investigate the distribution of the grouped data, i.e. distribution of places having x sign ins in our sub square.

In [ ]:
# distribution of grouped data
grouped_id = df_sub.groupby("place_id").count().sort_values(by="row_id")
grouped_id = grouped_id[grouped_id["row_id"]!=0]
print(grouped_id)


# grouped_id
sns.displot(data = grouped_id, x="row_id", bins=50)
sns.displot(data=grouped_id, x="row_id", kind="kde")

It becomes visible that there is a majority of place_id's that receive very few check ins, while a few place_ids contain the majority of the check ins.

The distribution could be similar to poisson or pareto distribution.




## K-Nearest-Neighbours

As a first classification algorithm, we will run a KNN algorithm on the sub-square of data using a grid of k parameters. This might be used as benchmark for later classification approaches.

The following cell contains an **algorithm to classify the examples within that sub-square of the grid** using a validation set.

### Data preparation for classification algorithm

Since there are extremely many classes with very few check_ins, for the classification model we will **remove** the examples which belong to **classes that are only classified a few times *th***. The magnitude of ***th*** will be treated as a hyperparameter.

In addition, the algorithm will also **remove** examples from the training which have an **accuracy** rating that is **above the given threshold *o***, so that examples with a rather uncertain classification are not trained on.

In [ ]:
# analysis function
def print_largest_entries(dict_, n):
    '''
    Return n largest key value pairs from dict
    '''
    n_largest = sorted(dict_.items(), key=lambda pair: pair[1], reverse=True )[:n]
    
    pprint("5 Combinations with highest prediction accuracy: ")
    print(n_largest)
    
    return n_largest

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from pprint import *
import warnings
warnings.filterwarnings('ignore')

# define function to perform classification on a subsquare of the grid, initialized by a grid ID (will be defined later on)
def process_subsquare(df_train, th, n, o):
    """   
    Classification inside one grid cell.
    """  
    
    #Working on df_train
    place_counts = df_train.place_id.value_counts()
    mask = (place_counts[df_train.place_id.values] >= th).values
    df_train = df_train.loc[mask]
    
    df_train = df_train[df_train["accuracy"] < o]
    

    #Working on df_test
    #df_cell_test = df_test.loc[df_test.grid_cell == grid_id]
    row_ids = df_test.index
    
    #Preparing data
    le = LabelEncoder()
    
    X = df_train.drop(['place_id'], axis=1).values.astype(int)
    y = le.fit_transform(df_train.place_id.values)
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.95, random_state = 999)
    
    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=n, weights='distance', metric='euclidean')
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict_proba(X_test)
    y_predi = np.argmax(y_pred, axis = 1)
    
    pred_labels = le.inverse_transform(np.argsort(y_pred, axis=1)[:,::-1][:,:1])
    acc = round(sklearn.metrics.accuracy_score(y_predi, y_test),2)
    return pred_labels, row_ids, acc
   


# predict for different number of neighbours and different number of classes:
dict_acc= {}
for n in (100,150,200,250,300,350,400,450,500):
    for m in (50,100,200,250,300,350,400,450,500): #,3,4,5,6,7,8,9,10,20,30
        for o in (15,25,50,100): # 150,200,300,400
            pred_labels, row_ids, acc = process_subsquare(df_sub, th = m, n = n, o = o)
            dict_acc[(n,m,o)] = acc




# sort dictionary
n_largest = print_largest_entries(dict_acc, 5)            
       
print("Max. accuracy: ", max(dict_acc, key=dict_acc.get), max(dict_acc.values()))



### Results from the subsquare analysis

The prediction accuracy using the validation set approach is rather high given its simplicity (maximum of **46%** for a combination of ). It shall be noted that in this example only one feature label is assigned to each example rather than 3, as mentioned in the task description. 

I will now parse the entire set by dividing it cell by cell, using a modified version of this parsing script: https://www.kaggle.com/svpons/grid-plus-classifier

The classification of the full dataset will be performed by using the hyperparameters of the test run that yielded the highest accuracy.

- k (number of nearest neighbours) = 200
- th (Minimum number of training set class assignments for class to be valid) = 500
- o (Maximum accuracy (error) value for training example to be kept) = 15




## Application of KNN to the full dataset

The parsing of the full dataset will be performed using a splitting algorithm for splitting the dataset into cells and applying the classification for each cell.

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

def prepare_data(df, n_cell_x, n_cell_y):
    """
    Computation of the grid.
    """
    #Creating the grid
    size_x = 10. / n_cell_x
    size_y = 10. / n_cell_y
    eps = 0.00001  
    xs = np.where(df.x.values < eps, 0, df.x.values - eps)
    ys = np.where(df.y.values < eps, 0, df.y.values - eps)
    pos_x = (xs / size_x).astype(np.int)
    pos_y = (ys / size_y).astype(np.int)
    df['grid_cell'] = pos_y * n_cell_x + pos_x
    
    #Feature engineering
    fw = [500, 1000, 4, 3, 1./22., 2, 10] #feature weights (black magic here)
    df.x = df.x.values * fw[0]
    df.y = df.y.values * fw[1]
    initial_date = np.datetime64('2014-01-01T01:01') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    df['hour'] = d_times.hour * fw[2]
    df['weekday'] = d_times.weekday * fw[3]
    df['day'] = (d_times.dayofyear * fw[4]).astype(int)
    df['month'] = d_times.month * fw[5]
    df['year'] = (d_times.year - 2013) * fw[6]

    df = df.drop(['time'], axis=1) 
    return df


def process_one_cell(df_train, df_test, grid_id, th):
    """   
    Classification inside one grid cell.
    """  
    
    o = 15
    
    #Working on df_train
    df_cell_train = df_train.loc[df_train.grid_cell == grid_id]
    place_counts = df_cell_train.place_id.value_counts()
    mask = (place_counts[df_cell_train.place_id.values] >= th).values
    df_cell_train = df_cell_train.loc[mask]
    df_cell_train = df_cell_train[df_cell_train["accuracy"] < o]
    

    #Working on df_test
    df_cell_test = df_test.loc[df_test.grid_cell == grid_id]
    row_ids = df_cell_test.index
    
    #Preparing data
    le = LabelEncoder()
    y = le.fit_transform(df_cell_train.place_id.values)
    print(y)
    X = df_cell_train.drop(['place_id', 'grid_cell'], axis=1).values.astype(int)
    print(X)
    X_test = df_cell_test.drop(['grid_cell'], axis = 1).values.astype(int)
    print(X_test)
    
    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=200, weights='distance', 
                               metric='manhattan')
    clf.fit(X, y)
    y_pred = clf.predict_proba(X_test)
    pred_labels = le.inverse_transform(np.argsort(y_pred, axis=1)[:,::-1][:,:3])
    return pred_labels, row_ids



def process_grid(df_train, df_test, th, n_cells):
    """
    Iterates over all grid cells, aggregates the results and makes the
    submission.
    """ 
    preds = np.zeros((df_test.shape[0], 3), dtype=int)
    print(preds)
    
    for g_id in range(n_cells):
        if g_id % 100 == 0:
            print('iter: %s' %(g_id))
        
        #Applying classifier to one grid cell
        pred_labels, row_ids = process_one_cell(df_train, df_test, g_id, th)

        #Updating predictions
        print("preds[row_ids]:", preds[row_ids])
        
        preds[row_ids] = pred_labels

    print('Generating submission file ...')
    #Auxiliary dataframe with the 3 best predictions for each sample
    df_aux = pd.DataFrame(preds, dtype=str, columns=['l1', 'l2', 'l3'])  
    
    #Concatenating the 3 predictions for each sample
    ds_sub = df_aux.l1.str.cat([df_aux.l2, df_aux.l3], sep=' ')
    
    #Writting to csv
    ds_sub.name = 'place_id'
    ds_sub.to_csv('sub_knn.csv', index=True, header=True, index_label='row_id')  
      

#Defining the size of the grid
n_cell_x = 20
n_cell_y = 40 

print('Preparing train data')
df_train_ = prepare_data(df_train, n_cell_x, n_cell_y)

print('Preparing test data')
df_test_ = prepare_data(df_test, n_cell_x, n_cell_y)

#Solving classification problems inside each grid cell
th = 500 #Keeping place_ids with more than th samples.   
#process_grid(df_train_, df_test_, th, n_cell_x*n_cell_y)

print('Generating submission file ...')
#df_submission.to_csv("sub.csv", index=True) 

## Gradient Boosting

Since we have now seen the accuracy and classification results from the cell-wise KNN algorithm, the approach is now to apply the cell-wise parsing of the data grid and to analyze each cell and predict the labels using a gradient boosting classifier.

For this example, I will use the XGBClassifier from the XGB toolbox. Again, I am using this parsing algorithm: https://www.kaggle.com/svpons/grid-plus-classifier


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from pprint import *

import xgboost as xgb

xgb_cl = xgb.XGBClassifier()

print(type(xgb_cl))


# define function to perform classification on a subsquare of the grid, initialized by a grid ID (will be defined later on)
def process_subsquare_xgb(df_train, th, o):
    """   
    Classification inside one grid cell.
    """  
    print(th, o)
    #Working on df_train
    place_counts = df_train.place_id.value_counts()
    mask = (place_counts[df_train.place_id.values] >= th).values
    df_train = df_train.loc[mask]
    
    df_train = df_train[df_train["accuracy"] < o]
    

    #Working on df_test
    #df_cell_test = df_test.loc[df_test.grid_cell == grid_id]
    row_ids = df_test.index
    
    #Preparing data
    le = LabelEncoder()
    
    X = df_train.drop(['place_id'], axis=1).values.astype(int)
    y = le.fit_transform(df_train.place_id.values)
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.95, random_state=999)
    
    
#     #Applying the classifier
#     clf = KNeighborsClassifier(n_neighbors=n, weights='distance', metric='euclidean')
#     clf.fit(X_train, y_train)
    
#     y_pred = clf.predict_proba(X_test)
#     y_predi = np.argmax(y_pred, axis = 1)
    
#     pred_labels = le.inverse_transform(np.argsort(y_pred, axis=1)[:,::-1][:,:1])
#     acc = round(sklearn.metrics.accuracy_score(y_predi, y_test),2)
#     return pred_labels, row_ids, acc
    
    
    
    #Applying the classifier
    xgb_cl = xgb.XGBClassifier()    
    xgb_cl.fit(X_train, y_train)
    
    y_pred = xgb_cl.predict(X_test)
    
    #pred_labels = le.inverse_transform(np.argsort(y_pred, axis=1)[:,::-1][:,:1])
    acc = round(metrics.accuracy_score(y_pred, y_test),2)
    return acc
    # return pred_labels, row_ids, acc



# predict for different number of neighbours and different number of classes:
dict_acc_xgb= {}
for m in (350,400,450,500): #,3,4,5,6,7,8,9,10,20,30
    for o in (10,12,15,25,50): # 150,200,300,400
        acc = process_subsquare_xgb(df_sub, th = m, o = o)
        dict_acc_xgb[(m,o)] = acc




# sort dictionary
n_largest = print_largest_entries(dict_acc_xgb, 5)            
       
print("Max. accuracy: ", max(dict_acc_xgb, key=dict_acc_xgb.get), max(dict_acc_xgb.values()))

### Results from the subsquare analysis (XGB)

The prediction accuracy using the validation set approach is high given the number of classes (maximum of up to **75%**). It shall be noted that in this example only one feature label is assigned to each example rather than 3, as mentioned in the task description. 

I will now parse the entire set by dividing it cell by cell, using a modified version of this parsing script: https://www.kaggle.com/svpons/grid-plus-classifier

The classification of the full dataset will be performed by using the hyperparameters of the test run that yielded the highest accuracy.

- th (Minimum number of training set class assignments for class to be valid) = 450
- o (Maximum accuracy (error) value for training example to be kept) = 15




## Application of XGB classifier to the full dataset

The parsing of the full dataset will be performed using a splitting algorithm for splitting the dataset into cells and applying the classification for each cell.

In [ ]:
# coding: utf-8
__author__ = 'Sandro Vega Pons : https://www.kaggle.com/svpons'

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
import xgboost as xgb

xgb_cl = xgb.XGBClassifier()

def prepare_data(df_train, df_test, n_cell_x, n_cell_y):
    """
    Some feature engineering (mainly with the time feature) + normalization 
    of all features (substracting the mean and dividing by std) +  
    computation of a grid (size = n_cell_x * n_cell_y), which is included
    as a new column (grid_cell) in the dataframes.
    
    Parameters:
    ----------    
    df_train: pandas DataFrame
              Training data
    df_test : pandas DataFrame
              Test data
    n_cell_x: int
              Number of grid cells on the x axis
    n_cell_y: int
              Number of grid cells on the y axis
    
    Returns:
    -------    
    df_train, df_test: pandas DataFrame
                       Modified training and test datasets.
    """  
    print('Feature engineering...')
    print('    Computing some features from x and y ...')
    ##x, y, and accuracy remain the same
        ##New feature x/y
    eps = 0.00001  #required to avoid some divisions by zero.
    df_train['x_d_y'] = df_train.x.values / (df_train.y.values + eps) 
    df_test['x_d_y'] = df_test.x.values / (df_test.y.values + eps) 
        ##New feature x*y
    df_train['x_t_y'] = df_train.x.values * df_train.y.values  
    df_test['x_t_y'] = df_test.x.values * df_test.y.values
    
    print('    Creating datetime features ...')
    ##time related features (assuming the time = minutes)
    initial_date = np.datetime64('2014-01-01T01:01') 
        #working on df_train  
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df_train.time.values)    
    df_train['hour'] = d_times.hour
    df_train['weekday'] = d_times.weekday
    df_train['day'] = d_times.day
    df_train['month'] = d_times.month
    df_train['year'] = d_times.year
    df_train = df_train.drop(['time'], axis=1)
        #working on df_test    
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df_test.time.values)    
    df_test['hour'] = d_times.hour
    df_test['weekday'] = d_times.weekday
    df_test['day'] = d_times.day
    df_test['month'] = d_times.month
    df_test['year'] = d_times.year
    df_test = df_test.drop(['time'], axis=1)
    
    print('Computing the grid ...')
    #Creating a new colum with grid_cell id  (there will be 
    #n = (n_cell_x * n_cell_y) cells enumerated from 0 to n-1)
    size_x = 10. / n_cell_x
    size_y = 10. / n_cell_y
        #df_train
    xs = np.where(df_train.x.values < eps, 0, df_train.x.values - eps)
    ys = np.where(df_train.y.values < eps, 0, df_train.y.values - eps)
    pos_x = (xs / size_x).astype(np.int)
    pos_y = (ys / size_y).astype(np.int)
    df_train['grid_cell'] = pos_y * n_cell_x + pos_x
            #df_test
    xs = np.where(df_test.x.values < eps, 0, df_test.x.values - eps)
    ys = np.where(df_test.y.values < eps, 0, df_test.y.values - eps)
    pos_x = (xs / size_x).astype(np.int)
    pos_y = (ys / size_y).astype(np.int)
    df_test['grid_cell'] = pos_y * n_cell_x + pos_x 
    
    ##Normalization
    print('Normalizing the data: (X - mean(X)) / std(X) ...')
    cols = ['x', 'y', 'accuracy', 'x_d_y', 'x_t_y', 'hour', 
            'weekday', 'day', 'month', 'year']
    for cl in cols:
        ave = df_train[cl].mean()
        std = df_train[cl].std()
        df_train[cl] = (df_train[cl].values - ave ) / std
        df_test[cl] = (df_test[cl].values - ave ) / std
        
    #Returning the modified dataframes
    return df_train, df_test


def process_one_cell(df_train, df_test, grid_id, th):
    """
    Does all the processing inside a single grid cell: Computes the training
    and test sets inside the cell. Fits a classifier to the training data
    and predicts on the test data. Selects the top 3 predictions.
    
    Parameters:
    ----------    
    df_train: pandas DataFrame
              Training set
    df_test: pandas DataFrame
             Test set
    grid_id: int
             The id of the grid to be analyzed
    th: int
       Threshold for place_id. Only samples with place_id with at least th
       occurrences are kept in the training set.
    
    Return:
    ------    
    pred_labels: numpy ndarray
                 Array with the prediction of the top 3 labels for each sample
    row_ids: IDs of the samples in the submission dataframe 
    """   
    #Working on df_train
    df_cell_train = df_train.loc[df_train.grid_cell == grid_id]
    place_counts = df_cell_train.place_id.value_counts()
    mask = place_counts[df_cell_train.place_id.values] >= th
    df_cell_train = df_cell_train.loc[mask.values]
    
    o = 15
    df_train = df_train[df_train["accuracy"] < o]
    
    #Working on df_test
    df_cell_test = df_test.loc[df_test.grid_cell == grid_id]
    row_ids = df_cell_test.index
    
    le = LabelEncoder()
    y = le.fit_transform(df_cell_train.place_id.values)
    X = df_cell_train.drop(['place_id', 'grid_cell'], axis = 1).values

    #Training Classifier
    xgb_cl = xgb.XGBClassifier()  
    xgb_cl.fit(X, y)
    X_test = df_cell_test.drop(['grid_cell'], axis = 1).values
    y_pred = xgb_cl.predict_proba(X_test)

    pred_labels = le.inverse_transform(np.argsort(y_pred, axis=1)[:,::-1][:,:3])    
    return pred_labels, row_ids
   
   
def process_grid(df_train, df_test, df_sub, th, n_cells):
    """
    Iterates over all grid cells and aggregates the results of individual cells
    """    
    for g_id in range(n_cells):
        if g_id % 10 == 0:
            print('iteration: %s' %(g_id))
        
        #Applying classifier to one grid cell
        pred_labels, row_ids = process_one_cell(df_train, df_test, g_id, th)
        #Converting the prediction to the submission format
        str_labels = np.apply_along_axis(lambda x: ' '.join(x.astype(str)), 
                                         1, pred_labels)
        #Updating submission file
        df_sub.loc[row_ids] = str_labels.reshape(-1,1)
        
    return df_sub       
                 


#Defining the size of the grid
n_cell_x = 10
n_cell_y = 10 
df_train, df_test = prepare_data(df_train, df_test, n_cell_x, n_cell_y)

#Solving classification problems inside each grid cell
th = 450 #Threshold on place_id inside each cell. Only place_ids with at 
        #least th occurrences inside each grid_cell are considered. This
        #is to avoid classes with very few samples and speed-up the 
        #computation.

#df_submission  = process_grid(df_train, df_test, df_sub, th = 450, n_cell_x * n_cell_y)                                 
#creating the submission
print('Generating submission file ...')
#df_submission.to_csv("sub.csv", index=True) 

## Summary of results

In summary, the classification approach using gradient boosting (XGB) worked well for the data, especially if the quantity of possible classes was reduced to a number of classes with many check ins. This way, the class assignment distortion could be reduced. In addition, reducing the training data by the training examples that had a low classification confidence (high accuracy value) improved the classification further, up to an accuracy of 75 % in the training subsquare using gradient boosting. 